In [1]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from ieeg.auth import Session
from scipy import signal as sig
import os
from os.path import join as ospj
from os.path import exists as ospe
import pathlib
from tqdm import tqdm

In [2]:
import sys
sys.path.append('/users/wojemann/iEEG_processing')
from pioneer import Pioneer

In [8]:
with open('config.json','r') as f:
    config = json.load(f)

In [9]:
ptList = config['patients']

In [10]:
display(config['paths'])
raw_datapath = config['paths']['RAW_DATA']
usr = config['paths']['iEEG_USR']
pwd = config['paths']['iEEG_PWD']
ignore_list = config['ignore']

{'META': '/mnt/leif/littlab/users/wojemann/stim-seizures/',
 'RAW_DATA': '/mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA',
 'PROCESSED_DATA': '/mnt/leif/littlab/users/wojemann/stim-seizures/PROCESSED_DATA',
 'iEEG_USR': 'wojemann',
 'iEEG_PWD': '/mnt/leif/littlab/users/wojemann/woj_ieeglogin.bin'}

In [11]:
ptList

['HUP170_phaseII',
 'HUP172_phaseII',
 'HUP178_phaseII',
 'HUP180_phaseII',
 'HUP181_phaseII_D01',
 'HUP181_phaseII_D02',
 'HUP186_phaseII',
 'HUP188_phaseII',
 'HUP193_phaseII_D01',
 'HUP193_phaseII_D02',
 'HUP193_phaseII_D03',
 'HUP194_phaseII_D01',
 'HUP194_phaseII_D02',
 'HUP194_phaseII_D03',
 'HUP197_phaseII_D01',
 'HUP197_phaseII_D02',
 'HUP201_phaseII_D01',
 'HUP201_phaseII_D02',
 'HUP202_phaseII',
 'HUP205_phaseII',
 'HUP205_phaseII',
 'HUP207_phaseII',
 'HUP210_phaseII',
 'HUP211_phaseII',
 'HUP213_phaseII_D01',
 'HUP213_phaseII_D02',
 'HUP214_phaseII_D01',
 'HUP216_phaseII_D02',
 'HUP218_phaseII_D01',
 'HUP218_phaseII_D02',
 'HUP224_phaseII',
 'HUP225_phaseII',
 'HUP230_phaseII',
 'HUP235_phaseII',
 'HUP238_phaseII',
 'HUP247_phaseII',
 'HUP249_phaseII',
 'HUP250_phaseII',
 'HUP253_phaseII',
 'HUP257_phaseII',
 'HUP224_CCEP',
 'HUP225_CCEP',
 'HUP230_CCEP',
 'HUP235_CCEP',
 'HUP238_CCEP',
 'HUP247_CCEP',
 'HUP249_CCEP',
 'HUP250_CCEP',
 'HUP253_CCEP',
 'HUP257_CCEP']

In [13]:
for i,pt in enumerate(ptList[:31]):
# for i,pt in enumerate(['EMU1591_Day02_1']):
    save_path = ospj(raw_datapath,pt[:6])
    print(pt,save_path)
    if not ospe(save_path):
        os.makedirs(save_path)
    wagon = Pioneer(usr,pwd,pt)
    wagon.pull_annotations()
    wagon.filter_seizure_annotations()
    # wagon.seizure_annotations.loc[:,'ID'] = wagon.ID
    wagon.seizure_annotations.to_csv(ospj(save_path,f'seizure_annotations_{pt}.csv'))

HUP170_phaseII /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP170
got 100 annotations on call # 1 covering 6064472656 usec to 120562132812 usec
got 100 annotations on call # 2 covering 120563718750 usec to 337410978515 usec
got 14 annotations on call # 3 covering 349083917968 usec to 453177324218 usec
Filtered       7.48% of all annotations
HUP172_phaseII /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP172
got 6 annotations on call # 1 covering 402704260829 usec to 692937000000 usec
Filtered       1.67% of all annotations
HUP178_phaseII /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP178
got 100 annotations on call # 1 covering 5819898437 usec to 309199076171 usec
got 57 annotations on call # 2 covering 309200380859 usec to 609991347656 usec
Filtered       7.71% of all annotations
HUP180_phaseII /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP180
got 100 annotations on call # 1 covering 9029806640 usec to 342387083984 usec
got 100 annotation

#### Goal of next section is to make a csv containing all of the seizure times for each of these patients. The csv should contain HUPID, EEC, UEO, flag for stim or not, 